# First part - Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [39]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [40]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [41]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [42]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [43]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [45]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [54]:
# replacing the \n for nothing in the df

import numpy

def reemplazar(x):
    return x.replace('\n', '')

df.columns = ['Postal code', 'Borough', 'Neighborhood']
df = df.applymap(reemplazar)

In [68]:
# Correcting Neighborhoods '/' for ','

def reemplazar(x):
    return x.replace(' /', ',')

df = df.applymap(reemplazar)

In [69]:
print(df)

    Postal code                Borough  \
2           M3A             North York   
3           M4A             North York   
4           M5A       Downtown Toronto   
5           M6A             North York   
6           M7A       Downtown Toronto   
8           M9A              Etobicoke   
9           M1B            Scarborough   
11          M3B             North York   
12          M4B              East York   
13          M5B       Downtown Toronto   
14          M6B             North York   
17          M9B              Etobicoke   
18          M1C            Scarborough   
20          M3C             North York   
21          M4C              East York   
22          M5C       Downtown Toronto   
23          M6C                   York   
26          M9C              Etobicoke   
27          M1E            Scarborough   
30          M4E           East Toronto   
31          M5E       Downtown Toronto   
32          M6E                   York   
36          M1G            Scarbor

In [71]:
# droping not assigned

# axis = 1 colums
# axis = 0 index

# Names of indexes where Borough == 'Not assigned'
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete rows
df.drop(indexNames , inplace=True)

In [72]:
# combine rows will be same postalcode, combining into one row with the neighborhoods separated with a comma

result = df.groupby(['Postal code','Borough'], sort=False).agg( ', '.join)

In [73]:
df_new=result.reset_index()

In [74]:
df_new.head(15)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [75]:
df.shape

(104, 3)